# Present the symbolic derivation of existing models

In [ ]:
%matplotlib widget
import os, importlib, inspect
from IPython.display import display, Math, Latex, Markdown
from bmcs_matmod.gsm.gsm_models.gsm_base import GSMBase, sp
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
modules = []

# Collect all relevant modules and their classes
for file_name in os.listdir('.'):
    if file_name.endswith('.py') and file_name not in ['gsm_base.py', '__init__.py']:
        module_path = f"bmcs_matmod.gsm.gsm_models.{file_name[:-3]}"
        module = importlib.import_module(module_path)
        for name, obj in inspect.getmembers(module, inspect.isclass):
            if issubclass(obj, GSMBase) and obj is not GSMBase:
                modules.append((name, obj))

# Sort the modules by class names
modules.sort(key=lambda x: x[0])

gsm_models = {name: obj() for name, obj in modules}
gsm_models

In [ ]:
mparams = dict(E = 20, K = 5, S = 2, c = 1, r = 1, eps_0 = 0, f_c = 15, eta_vp=10, eta_ve=100)
n_t = 151
n_I = 1
eps_max = 2.5
t_t = np.linspace(0, 10, n_t)
eps_n_t = np.linspace(0, eps_max, n_t)
response = {}
selection = [] # ['GSM1D_VEVP', 'GSM1D_VEVPD']
if len(selection) == 0:
    selection = gsm_models.keys()
for name in selection:
    gsm = gsm_models[name]
    print(f'name: {name}')
    response[name] = gsm.get_response(eps_n_t, t_t, **mparams)


In [ ]:
fig, ax_sig = plt.subplots(1, 1, figsize=(8, 5))

ax_iter = ax_sig.twinx()

for name, resp in response.items():
    t_t, eps_ta, sig_ta, Eps_t, Sig_t, iter_t, lam_t, (d_t_t, d_eps_ta) = resp
    print(iter_t.shape)
    eps_t = eps_ta[:, 0]
    iter_t = iter_t[:, 0]
    sig_t = sig_ta[:, 0, 0]
    ax_sig.plot(eps_t, sig_t, label=rf'{name}')
    ax_iter.plot(eps_t, iter_t, label=rf'{name}', ls='dashed')
    # Add a box with the material parameters
    param_text = ', '.join([f'{key}: {value}' for key, value in mparams.items()])
    param_text = '\n'.join([', '.join(param_text.split(', ')[i:i+5]) for i in range(0, len(param_text.split(', ')), 5)])
    props = dict(boxstyle='round', facecolor='lightgray', edgecolor='lightgray', alpha=0.5)
    ax_sig.text(0.5, 0.97, param_text, transform=ax_sig.transAxes, fontsize=11,
                verticalalignment='top', horizontalalignment='center', bbox=props)
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend(fontsize='small')


In [ ]:
mparams = dict(E = 20, K = 5, S = 2, c = 1, r = 1, eps_0 = 0, f_c = 15, eta_vp=10, eta_ve=100)
n_t = 151
n_I = 1
sig_max = 20
t_t = np.linspace(0, 10, n_t)
sig_n_t = np.linspace(0, sig_max, n_t)
response = {}
selection = ['GSM1D_EVP']
if len(selection) == 0:
    selection = gsm_models.keys()
for name in selection:
    gsm = gsm_models[name]
    print(f'name: {name}')
    response[name] = gsm.get_G_response(sig_n_t, t_t, **mparams)


In [ ]:
fig, ax_sig = plt.subplots(1, 1, figsize=(8, 5))

ax_iter = ax_sig.twinx()

for name, resp in response.items():
    t_t, sig_ta, eps_ta, Eps_t, Sig_t, iter_t, lam_t, (d_t_t, d_eps_ta) = resp
    print(iter_t.shape)
    eps_t = eps_ta[:, 0]
    iter_t = iter_t[:, 0]
    sig_t = sig_ta[:, 0]
    ax_sig.plot(eps_t, sig_t, label=rf'{name}')
    ax_iter.plot(eps_t, iter_t, label=rf'{name}', ls='dashed')
    # Add a box with the material parameters
    param_text = ', '.join([f'{key}: {value}' for key, value in mparams.items()])
    param_text = '\n'.join([', '.join(param_text.split(', ')[i:i+5]) for i in range(0, len(param_text.split(', ')), 5)])
    props = dict(boxstyle='round', facecolor='lightgray', edgecolor='lightgray', alpha=0.5)
    ax_sig.text(0.5, 0.97, param_text, transform=ax_sig.transAxes, fontsize=11,
                verticalalignment='top', horizontalalignment='center', bbox=props)
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend(fontsize='small')
